In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow import keras
from tensorflow.keras.layers import Embedding, Dense, Flatten, Input, Concatenate
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler

In [ ]:
user_encoder = LabelEncoder()
survey_encoder = LabelEncoder()

In [ ]:
df = pd.read_csv('survey_recommendation_dataset.csv')

In [ ]:
df.head()

,UserID,PreviousSurveyType,ResponseTime (mins),SurveyDateTime,Rating (1-5)
0,101,Market Research,19,2024-01-01 00:00:00,5
1,167,Customer Satisfaction,11,2024-01-01 01:00:00,1
2,111,Product Feedback,16,2024-01-01 02:00:00,3
3,116,Employee Engagement,28,2024-01-01 03:00:00,1
4,153,Event Feedback,10,2024-01-01 04:00:00,1


In [ ]:
df["UserID"] = user_encoder.fit_transform(df["UserID"])
df["PreviousSurveyType"] = survey_encoder.fit_transform(df["PreviousSurveyType"])

In [ ]:
# Normalize numerical features
scaler = MinMaxScaler()
df["ResponseTime (mins)"] = scaler.fit_transform(df[["ResponseTime (mins)"]])
df["Rating (1-5)"] = scaler.fit_transform(df[["Rating (1-5)"]])

In [ ]:
# Define input features and target variable
X = df[["UserID", "PreviousSurveyType", "ResponseTime (mins)", "Rating (1-5)"]]
y = df["PreviousSurveyType"]  # Predicting recommended survey type

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [ ]:
# Define the deep learning model
input_user = Input(shape=(1,))
input_survey = Input(shape=(1,))
input_response_time = Input(shape=(1,))
input_rating = Input(shape=(1,))

In [ ]:
embedding_user = Embedding(input_dim=len(user_encoder.classes_), output_dim=10)(input_user)
embedding_survey = Embedding(input_dim=len(survey_encoder.classes_), output_dim=10)(input_survey)

In [ ]:
flat_user = Flatten()(embedding_user)
flat_survey = Flatten()(embedding_survey)

In [ ]:
concatenated = Concatenate()([flat_user, flat_survey, input_response_time, input_rating])
dense1 = Dense(64, activation="relu")(concatenated)
dense2 = Dense(32, activation="relu")(dense1)
output = Dense(len(survey_encoder.classes_), activation="softmax")(dense2)


In [ ]:
# Compile model
model = keras.Model(inputs=[input_user, input_survey, input_response_time, input_rating], outputs=output)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [ ]:
# Train the model
history = model.fit(
    [X_train["UserID"], X_train["PreviousSurveyType"], X_train["ResponseTime (mins)"], X_train["Rating (1-5)"]],
    y_train,
    epochs=10,
    batch_size=32,
    validation_data=(
        [X_test["UserID"], X_test["PreviousSurveyType"], X_test["ResponseTime (mins)"], X_test["Rating (1-5)"]],
        y_test
    )
)

Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 5s 21ms/step - accuracy: 0.2384 - loss: 1.7781 - val_accuracy: 0.4050 - val_loss: 1.7309
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - accuracy: 0.5185 - loss: 1.6876 - val_accuracy: 0.7150 - val_loss: 1.5080
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8047 - loss: 1.3846 - val_accuracy: 0.8650 - val_loss: 0.9714
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.9276 - loss: 0.7803 - val_accuracy: 1.0000 - val_loss: 0.3529
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.2382 - val_accuracy: 1.0000 - val_loss: 0.0826
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0578 - val_accuracy: 1.0000 - val_loss: 0.0258
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 1.0000 - loss: 0.0199 - val_accuracy: 1.0000 - val_loss: 0.0133
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 1.0000 - loss: 0.0108 - val_accuracy: 1.0000 - val_loss

In [ ]:
# Evaluate model
test_loss, test_accuracy = model.evaluate(
    [X_test["UserID"], X_test["PreviousSurveyType"], X_test["ResponseTime (mins)"], X_test["Rating (1-5)"]],
    y_test
)

test_accuracy

7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 1.0000 - loss: 0.0040 


1.0

In [ ]:
model.save("survey_recommendation_model.h5")
print("Model saved successfully!")

Model saved successfully!


In [ ]:
# Load the trained model
model = tf.keras.models.load_model("survey_recommendation_model.h5")

In [ ]:
test_data = np.array([
    [101, 2, 12.5, 4.0],  # (UserID, SurveyType, ResponseTime, Rating)
    [102, 1, 8.0, 5.0],
    [103, 3, 15.0, 3.0]
])

In [ ]:
scaler = MinMaxScaler()
test_data = scaler.fit_transform(test_data)  # Ensure to use the same scaler from training

In [ ]:
import numpy as np

# Example test input (UserID, SurveyType, ResponseTime, Rating)
test_data = np.array([[107, 3, 14.0, 4.2]])  # Ensure shape is (1, 4)

# Normalize test data using the same scaler from training
test_data = scaler.transform(test_data)

# Split data into separate inputs (assuming 4 input layers in the model)
input_1 = test_data[:, 0]  # UserID
input_2 = test_data[:, 1]  # SurveyType
input_3 = test_data[:, 2]  # ResponseTime
input_4 = test_data[:, 3]  # Rating

# Pass as a list to predict()
predictions = model.predict([input_1, input_2, input_3, input_4])

# Get the recommended survey ID
recommended_survey = np.argmax(predictions)
print(f"Recommended Survey ID: {recommended_survey}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
Recommended Survey ID: 1
